# Préparation des données

In [8]:
%run Récupération.ipynb

## 1 - Inventaire des données

Après la réunion avec Selim, il nous a donné une marche à suivre très précise qui est détaillé dans le fichier [Rapport](../rapports/Rapport_2411.md). La première étape est de faire un inventaire des variables et de créer une base de donnée les contenant tous. Il faut bien garder en tête que les différents modèles ne verront que ligne par ligne.


### Variables qualitatives

   Nom de la variable    | Abréviation
   ----------------------|:-----------:
   Identifiant du joueur |  PLAYER_ID
   Équipe du joueur      |    TEAM
   Équipe adverse        |     ADV
   Domicile / Exterieur  |    HOME
   Adversaire directe    |   ADV_ID
 
### Variables quantitiatives

#### Variables scores positives

   Nom de la variable               | Abréviation 
   ---------------------------------|:-----------:
   Nombre de points                 |     PTS    
   Nombre de rebonds                |     REB
   Nombre de passes décisives       |     AST
   Nombre de récupération de balle  |     STL
   Nombre de block                  |     BLK
   Nombre de shoot réussi           |     FGM
   Nombre de lancer franc réussi    |     FTM
   Nombre de tire à 3 points réussi |     FG3M

#### Variables scores négatives

   Nom de la variable                | Abréviation 
   ----------------------------------|:-----------:
   Nombre de perte de balle          |     TOV
   Nombre de tire raté               |     FGF
   Nombre de lancer franc raté       |     FTF
   Nombre de tire à 3 points de raté |     FG3F

#### Variables de scores globales

   Nom de la variable            | Abréviation
   ------------------------------|:-----------:
   Somme des variables positives |    BONUS
   Somme des variables négatives |    MALUS
   Score du joueur               |    SCORE   
   
#### Variables laggées   
   
Toutes les variables ci-dessus ne sont pas accéssible pour le temps $t$ donc nous utilisons ces variables :
   
   Nom de la variable                                  |      Abréviation
   ----------------------------------------------------|:--------------------:
   Moyenne des N derniers matchs                       |   AVG_[ABREVIATION]
   Moyenne des N derniers matchs contre l'équipe [ADV] | AVG_ADV\\_[ABREVIATION]
   La série de victoire                                |        NB_VIC 
   La série de victoire contre l'équipe [ADV]          |       NB_VIC\\_ADV
   
### Variables temporelles

   Nom de la variable | Abréviation
   -------------------|:-----------:
   Date du match      |  GAME_DATE
   
### Hyperparamètres

   Nom du paramètre                                                     | Abréviation
   ---------------------------------------------------------------------|:-----------:
   Le nombre de match que l'on retient pour les moyennes                |      N
   Le nombre de match que l'on retient pour les moyennes par adversaire |    N_ADD


In [10]:
SEASON = 2019

# Hyperparamètre : le nombre de match que l'on regarde en arrière
N = 3
N_ADD = 1

# On récupère les données que l'on a récupéré précédemment
df = score_df(pd.read_csv(f'../data/boxscore_{SEASON}.csv'))

In [19]:
# La liste des variables dont on va récupérer les moyennes
list_var_avg = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M',
                 'TOV', 'FGF', 'FG3F', 'FTF']

In [12]:
def avg_df(df, n=N):
    '''Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    intéressantes
    
    Args :
        df : database avec les variables dans list_var_avg
        
    '''
    
    def avg(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_')
                                
    
    # Calcul des moyennes pour chaque joueur puis on recréer une base
    df_mean = df.groupby('PLAYER_ID').apply(avg)  \
                                        .reset_index()  \
                                        .set_index('level_1')  \

    return pd.concat([df, df_mean.drop('PLAYER_ID', axis=1)], axis=1)

In [13]:
avg_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF,AVG_BONUS,AVG_MALUS,AVG_SCORE
6646,22019,203114,21900364,2019-12-11,MIL,NOP,True,True,24,5,4,2,0,11,1,1,1,11,3,0,48,15,33,18.000000,7.666667,4.666667,0.666667,0.333333,7.0,1.0,3.000000,1.333333,6.000000,2.666667,0.000000,42.333333,10.000000,32.333333
19821,22019,1629011,21900072,2019-11-01,NYK,BOS,False,False,6,6,0,0,2,3,0,0,1,2,0,1,17,4,13,13.000000,6.666667,0.666667,0.000000,2.666667,5.0,3.0,0.000000,0.333333,2.333333,0.000000,0.666667,31.000000,3.333333,27.666667
9631,22019,203924,21900353,2019-12-10,DEN,PHI,False,False,3,1,3,2,2,1,0,1,0,5,2,0,13,7,6,11.666667,3.000000,0.666667,0.333333,0.666667,4.0,1.0,2.666667,2.000000,3.333333,2.333333,1.333333,24.000000,9.000000,15.000000


In [14]:
def avg_add_df(df, n=N_ADD):
    # Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    # intéressantes
    def avg_add(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_ADD_')
    
    # Calcul des moyennes pour chaque joueur et pour chaque équipe 
    # puis on recréer une base
    df_add_mean = df.groupby('PLAYER_ID').apply(lambda x:
                                                x.groupby('ADV').apply(avg_add))  \
                                            .reset_index()  \
                                            .set_index('level_2')  \
    
    return pd.concat([df, df_add_mean.drop(['PLAYER_ID', 'ADV'], axis=1)]  \
                                                                      , axis=1)

In [15]:
# TRÈS LONG !!!
#avg_add_df(df).sample(3)

In [16]:
def nb_vic_df(df):
    
    # Code non vectoriel : 
    def nb_vic(df):
        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]

        return pd.Series(serie, index=df.index, name='NB_VIC').to_frame()

    # On calcul les séries pour chaque joueur
    df_nb_vic = df.groupby('PLAYER_ID').apply(nb_vic)

    # On rajoute les séries à la fin de notre dataframe
    return pd.concat([df, df_nb_vic], axis=1)

In [17]:
nb_vic_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,NB_VIC
7345,22019,203463,21900806,2020-02-11,HOU,BOS,True,True,3,2,0,0,0,1,0,1,0,3,3,0,7,6,1,0
21665,22019,1629168,21901297,2020-08-11,ORL,BKN,True,False,8,7,1,1,0,2,4,0,2,6,1,0,23,9,14,0
9693,42019,203924,41900236,2020-09-13,DEN,LAC,True,True,4,0,2,0,1,0,4,0,0,3,1,0,11,4,7,1


In [18]:
def nb_vic_add_df(df):
    
    # Code non vectoriel : 
    def nb_vic_add(df):

        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]
        return pd.Series(serie, index=df.index, name='NB_VIC_ADD').to_frame()

    # On calcul les séries pour chaque joueur et pour chaque équipe
    df_nb_vic_add = df.groupby('PLAYER_ID').apply(lambda df: 
                        df.groupby('ADV').apply(nb_vic_add))  \
                                            .reset_index()  \
                                            .set_index('level_1')  \
                                            .drop('PLAYER_ID', axis=1)

    return pd.concat([df, df_nb_vic_add], axis=1)

In [52]:
# TRÈS LONG !!
#nb_vic_add_df(df)

In [53]:
# TRÈS LONG !!!
# La dataframe finale  

#df = nb_vic_add_df(nb_vic_df(avg_add_df(avg_df(df))))

#df.to_csv('../data/boxscore_2019_final.csv', index=False)